In [63]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.impute import KNNImputer

Read in and clean data

In [195]:
dropRows = [55, 76, 78, 81, 410, 481, 486, 490, 645, 653, 657, 693,
            697, 840, 827, 830, 842, 844, 949, 950, 1059, 1063, 1077, 1079, 1081]
#original data set had nonsense rows

superheroData = \
    pd.read_excel("Superhero data.xlsx").\
    drop(dropRows).\
    drop(["full_name"], axis = 1).\
    dropna(how = "all").\
    reset_index()#.\
    #drop(["index"], axis = 1)

superheroData["height"] = superheroData["height"].replace(',', '')
superheroData["weight"] = superheroData["weight"].replace(',', '')


superheroData

,index,name,real_name,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,Unnamed: 68
0,0,3 D Man,"Delroy Garrett, Jr.",85,30,60,60,40,70,"['Super Speed', 'Super Strength']",...,0,0,0,0,1,0,0,0,1,NaN
1,1,514A (Gotham),Bruce Wayne,100,20,30,50,35,100,"['Durability', 'Reflexes', 'Super Strength']",...,0,0,0,1,0,1,0,0,1,NaN
2,2,A Bomb,Richard Milhouse Jones,80,100,80,100,100,80,"['Accelerated Healing', 'Agility', 'Berserk Mo...",...,1,0,0,1,1,1,1,1,1,NaN
3,3,Aa,Aa,80,50,55,45,100,55,"['Energy Absorption', 'Energy Armor', 'Energy ...",...,0,0,0,0,0,0,0,0,0,NaN
4,4,Aaron Cash,Aaron Cash,80,10,25,40,30,50,"['Weapon based Powers', 'Weapons Master']",...,0,1,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,1428,Zane,Zane,100,35,50,60,100,100,"['Agility', 'Animal Control', 'Cold Resistance...",...,0,1,1,1,0,1,1,1,1,NaN
1404,1429,Zatanna,Zatanna Zatara,90,10,25,30,100,55,"['Cryokinesis', 'Fire Control', 'Magic', 'Prob...",...,0,0,0,0,0,0,0,0,0,NaN
1405,1430,Zoom (New 52),Hunter Zolomon,95,50,100,75,100,80,"['Accelerated Healing', 'Agility', 'Durability...",...,1,0,1,1,1,1,1,1,1,NaN
1406,1431,Zoom,Hunter Zolomon,75,10,100,30,100,30,"['Intangibility', 'Super Speed', 'Time Manipul...",...,0,0,0,0,1,0,0,0,0,NaN


In [196]:
superheroData.isnull().sum()

index                    0
name                     0
real_name              137
intelligence_score      16
strength_score          16
                      ... 
has_durability          78
has_stamina             78
has_agility             78
has_super_strength      78
Unnamed: 68           1408
Length: 69, dtype: int64

Fix format of height and weight variables:

In [197]:
columnLength = 1408

for i in range(0, columnLength - 1):
        #height data is of the form "6'8 â€¢ 203 cm" which would be very difficult to work with
        #hence I'm extracting just the value in centimeters and converting to a float
        #similar for weight
        if not(str(superheroData["height"][i]) == "nan" or str(superheroData["height"][i]) == ""):
            superheroData["height"][i] = float(str(superheroData["height"][i]).split()[2])

        if not(str(superheroData["weight"][i]) == "nan" or str(superheroData["weight"][i]) == ""):
            superheroData["weight"][i] = float(str(superheroData["weight"][i]).split()[3])

superheroData

IndexError: list index out of range

In [198]:
print(superheroData["height"][5])

#superheroData["height"][1422]=float(str(superheroData["height"][1422]).split()[2])
str(superheroData["height"][5]) == "nan"

superheroData

,index,name,real_name,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,...,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,Unnamed: 68
0,0,3 D Man,"Delroy Garrett, Jr.",85,30,60,60,40,70,"['Super Speed', 'Super Strength']",...,0,0,0,0,1,0,0,0,1,NaN
1,1,514A (Gotham),Bruce Wayne,100,20,30,50,35,100,"['Durability', 'Reflexes', 'Super Strength']",...,0,0,0,1,0,1,0,0,1,NaN
2,2,A Bomb,Richard Milhouse Jones,80,100,80,100,100,80,"['Accelerated Healing', 'Agility', 'Berserk Mo...",...,1,0,0,1,1,1,1,1,1,NaN
3,3,Aa,Aa,80,50,55,45,100,55,"['Energy Absorption', 'Energy Armor', 'Energy ...",...,0,0,0,0,0,0,0,0,0,NaN
4,4,Aaron Cash,Aaron Cash,80,10,25,40,30,50,"['Weapon based Powers', 'Weapons Master']",...,0,1,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,1428,Zane,Zane,100,35,50,60,100,100,"['Agility', 'Animal Control', 'Cold Resistance...",...,0,1,1,1,0,1,1,1,1,NaN
1404,1429,Zatanna,Zatanna Zatara,90,10,25,30,100,55,"['Cryokinesis', 'Fire Control', 'Magic', 'Prob...",...,0,0,0,0,0,0,0,0,0,NaN
1405,1430,Zoom (New 52),Hunter Zolomon,95,50,100,75,100,80,"['Accelerated Healing', 'Agility', 'Durability...",...,1,0,1,1,1,1,1,1,1,NaN
1406,1431,Zoom,Hunter Zolomon,75,10,100,30,100,30,"['Intangibility', 'Super Speed', 'Time Manipul...",...,0,0,0,0,1,0,0,0,0,NaN


Impute missing data values:

In [64]:
numericData = superheroData[superheroData.columns[]]

inputer = KNNImputer(n_neighbors = 10)
inputer.fit_transform(superheroData)

ValueError: could not convert string to float: '3-D Man'